In [1]:
str_pytket = """
OPENQASM 2.0;
include "qelib1.inc";

qreg qr[2];
creg cr[2];
u2(0.0*pi,1.0*pi) qr[0];
u2(0.0*pi,1.0*pi) qr[1];
cx qr[1],qr[0];
u1(1.74038062947225*pi) qr[0];
cx qr[1],qr[0];
u2(0.0*pi,1.0*pi) qr[0];
u2(0.0*pi,1.0*pi) qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
"""

str_qiskit = """
OPENQASM 2.0;
include "qelib1.inc";
qreg qr[2];
creg cr[2];
rxx(5.467567) qr[1],qr[0];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
"""

In [2]:
from qiskit.qasm2 import loads
from qiskit import qasm2

circuit_qiskit = loads(
    str_qiskit, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
print(circuit_qiskit)
circuit_pytket = loads(
    str_pytket, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
print(circuit_pytket)

      ┌──────────────┐┌─┐   
qr_0: ┤1             ├┤M├───
      │  Rxx(5.4676) │└╥┘┌─┐
qr_1: ┤0             ├─╫─┤M├
      └──────────────┘ ║ └╥┘
cr: 2/═════════════════╩══╩═
                       0  1 
      ┌─────────┐┌───┐┌────────────┐┌───┐┌─────────┐┌─┐   
qr_0: ┤ U2(0,π) ├┤ X ├┤ U1(5.4676) ├┤ X ├┤ U2(0,π) ├┤M├───
      ├─────────┤└─┬─┘└────────────┘└─┬─┘├─────────┤└╥┘┌─┐
qr_1: ┤ U2(0,π) ├──■──────────────────■──┤ U2(0,π) ├─╫─┤M├
      └─────────┘                        └─────────┘ ║ └╥┘
cr: 2/═══════════════════════════════════════════════╩══╩═
                                                     0  1 


In [5]:
from mqt import qcec
import tempfile

# create two temporary files
with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".qasm") as f_qiskit, tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".qasm") as f_pytket:
    f_qiskit.write(str_qiskit)
    f_pytket.write(str_pytket)
    f_qiskit.flush()
    f_pytket.flush()
    print(f_qiskit.name)
    print(f_pytket.name)

    # verify the two files
    res = qcec.verify(f_qiskit.name, f_pytket.name)
    equivalence = str(res.equivalence)
    print(f"Equivalence: {equivalence}")

/tmp/tmpfllnpr7d.qasm
/tmp/tmpj5yg0znj.qasm
Equivalence: equivalent_up_to_global_phase


In [18]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import Session, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import pandas as pd


def execute_circuit(qc: QuantumCircuit, platform_name: str):
    # Run the sampler job locally using AerSimulator.
    # Session syntax is supported but ignored.
    aer_sim = AerSimulator()

    # Transpile
    pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
    isa_qc = pm.run(qc)

    # The session is used but ignored in AerSimulator.
    sampler = Sampler(mode=aer_sim)
    result = sampler.run([isa_qc], shots=1000000).result()[0]

    # Section: Results
    counts = result.data.cr.get_counts()
    print(f"Measurement results: {counts}")
    # counts["platform_name"] = platform_name

    return counts


N_TESTS = 5
records_qiskit = []
records_pytket = []

for i in range(N_TESTS):
    print(f"Test {i}")
    print("Qiskit")
    res = execute_circuit(circuit_qiskit, "qiskit")
    records_qiskit.append(res)

    print("Pytket")
    res = execute_circuit(circuit_pytket, "pytket")
    records_pytket.append(res)

df_results_qiskit = pd.DataFrame.from_records(records_qiskit)
df_results_pytket = pd.DataFrame.from_records(records_pytket)

Test 0
Qiskit
Measurement results: {'00': 842491, '11': 157509}
Pytket
Measurement results: {'00': 842731, '11': 157269}
Test 1
Qiskit
Measurement results: {'00': 843610, '11': 156390}
Pytket
Measurement results: {'00': 843078, '11': 156922}
Test 2
Qiskit
Measurement results: {'00': 843333, '11': 156667}
Pytket
Measurement results: {'00': 842539, '11': 157461}
Test 3
Qiskit
Measurement results: {'00': 842397, '11': 157603}
Pytket
Measurement results: {'00': 843726, '11': 156274}
Test 4
Qiskit
Measurement results: {'00': 842425, '11': 157575}
Pytket
Measurement results: {'00': 842741, '11': 157259}


In [17]:
# get the average of the results for each platform
# and compare them

print("Qiskit")
print(df_results_qiskit.mean())

print("Pytket")
print(df_results_pytket.mean())

Qiskit
00    84254.6
11    15745.4
dtype: float64
Pytket
00    84342.2
11    15657.8
dtype: float64


In [20]:
from qiskit.quantum_info import Statevector

str_pytket = """
OPENQASM 2.0;
include "qelib1.inc";

qreg qr[2];
creg cr[2];
u2(0.0*pi,1.0*pi) qr[0];
u2(0.0*pi,1.0*pi) qr[1];
cx qr[1],qr[0];
u1(1.74038062947225*pi) qr[0];
cx qr[1],qr[0];
u2(0.0*pi,1.0*pi) qr[0];
u2(0.0*pi,1.0*pi) qr[1];
"""

str_qiskit = """
OPENQASM 2.0;
include "qelib1.inc";
qreg qr[2];
creg cr[2];
rxx(5.467567) qr[1],qr[0];
"""

circuit_qiskit = loads(
    str_qiskit, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
circuit_pytket = loads(
    str_pytket, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)


sv_qiskit = Statevector.from_instruction(circuit_qiskit)
print(sv_qiskit)

sv_pytket = Statevector.from_instruction(circuit_pytket)
print(sv_pytket)

# compare the statevectors
probabilities_qiskit = sv_qiskit.probabilities_dict()
print(probabilities_qiskit)
probabilities_pytket = sv_pytket.probabilities_dict()
print(probabilities_pytket)

Statevector([-0.91799191+0.j       ,  0.        +0.j       ,
              0.        +0.j       ,  0.        -0.3965991j],
            dims=(2, 2))
Statevector([8.42709152e-01-3.64074768e-01j,
             2.37807546e-17+8.32519166e-17j,
             2.21609759e-18+5.07638576e-17j,
             1.57290848e-01+3.64074768e-01j],
            dims=(2, 2))
{'00': 0.8427091522089964, '11': 0.15729084779100358}
{'00': 0.8427091522089958, '01': 7.496405907426081e-33, '10': 2.5818803232508824e-33, '11': 0.15729084779100347}


In [ ]:
from qiskit.quantum_info import Statevector

sv = Statevector.from_instruction(circuit_qiskit)
print(sv)

In [67]:
from pytket.passes import KAKDecomposition
from pytket.extensions.qiskit import qiskit_to_tk
from pytket.qasm import circuit_to_qasm_str
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

qc = QuantumCircuit(2, 2)
# qc.rzz(0.462419,0, 1)
qc.rxx(5.467567, 1, 0)
# qc.measure(0, 0)
# qc.measure(1, 1)

qc = qc.decompose(reps=2)
# from qiskit import transpile
# qc = transpile(qc, basis_gates=['u1', 'u2', 'cx'])
print(qc)


tket_circ = qiskit_to_tk(qc)
qasm_str_tket = circuit_to_qasm_str(tket_circ, header='qelib1',
                                    maxwidth=200)
print(qasm_str_tket)


# pass_ = KAKDecomposition()
# qc_after = pass_.apply(tket_circ)
# qasm_str_tket = circuit_to_qasm_str(qc_after, header='qelib1',
#                                     maxwidth=200)
# print(qasm_str_tket)

global phase: 3.5494
     ┌─────────┐┌───┐┌────────────┐┌───┐┌─────────┐
q_0: ┤ U2(0,π) ├┤ X ├┤ U1(5.4676) ├┤ X ├┤ U2(0,π) ├
     ├─────────┤└─┬─┘└────────────┘└─┬─┘├─────────┤
q_1: ┤ U2(0,π) ├──■──────────────────■──┤ U2(0,π) ├
     └─────────┘                        └─────────┘
c: 2/══════════════════════════════════════════════
                                                   
OPENQASM 2.0;
include "qelib1.inc";

qreg q[2];
creg c[2];
u2(0.0*pi,1.0*pi) q[0];
u2(0.0*pi,1.0*pi) q[1];
cx q[1],q[0];
u1(1.74038062947225*pi) q[0];
cx q[1],q[0];
u2(0.0*pi,1.0*pi) q[0];
u2(0.0*pi,1.0*pi) q[1];



In [41]:
circuit_single_decompose = loads(
    qasm_str_tket, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
sv_pytket_single_decompose = Statevector.from_instruction(
    circuit_single_decompose)

probabilities_pytket_single_decompose = sv_pytket_single_decompose.probabilities_dict()
print(probabilities_pytket_single_decompose)

{'00': 1.0}


In [53]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from math import sin, cos, pi


theta = 5.467567  # Example angle
matrix = [[cos(theta/2), 0, 0, -1j*sin(theta/2)],
          [0, cos(theta/2), -1j*sin(theta/2), 0],
          [0, -1j*sin(theta/2), cos(theta/2), 0],
          [-1j*sin(theta/2), 0, 0, cos(theta/2)]]
gate = UnitaryGate(matrix, label='rxx')

circuit = QuantumCircuit(2)
circuit.append(gate, [1, 0])

circuit = circuit.decompose(gates_to_decompose={'rxx'})
print(circuit)

sv_unitary_gate = Statevector.from_instruction(circuit)
probabilities_unitary_gate = sv_unitary_gate.probabilities_dict()
print(probabilities_unitary_gate)

     ┌──────┐
q_0: ┤1     ├
     │  rxx │
q_1: ┤0     ├
     └──────┘
{'00': 0.8427091522089964, '11': 0.15729084779100358}


In [68]:
from qiskit import transpile
qc_isa = transpile(circuit, basis_gates=['u1', 'u2', 'cx'])
print(qc_isa)

global phase: 0.2753
     ┌──────────────┐┌──────────┐┌─────────┐┌──────────┐┌──────────┐»
q_0: ┤ U1(-0.56139) ├┤ U1(-π/2) ├┤ U2(0,π) ├┤ U1(-π/2) ├┤ U1(3π/2) ├»
     └──┬───────┬───┘├──────────┤├─────────┤├──────────┤├──────────┤»
q_1: ───┤ U1(0) ├────┤ U1(-π/2) ├┤ U2(0,π) ├┤ U1(-π/2) ├┤ U1(3π/2) ├»
        └───────┘    └──────────┘└─────────┘└──────────┘└──────────┘»
«     ┌──────────┐┌─────────┐┌──────────┐┌────────┐┌───┐ ┌────────────┐ »
«q_0: ┤ U1(-π/2) ├┤ U2(0,π) ├┤ U1(-π/2) ├┤ U1(3π) ├┤ X ├─┤ U1(-2.243) ├─»
«     ├──────────┤├─────────┤├──────────┤├────────┤└─┬─┘┌┴────────────┴┐»
«q_1: ┤ U1(-π/2) ├┤ U2(0,π) ├┤ U1(-π/2) ├┤ U1(3π) ├──■──┤ U1(-0.21152) ├»
«     └──────────┘└─────────┘└──────────┘└────────┘     └──────────────┘»
«     ┌──────────┐┌─────────┐┌──────────┐┌────────────┐┌──────────┐┌─────────┐»
«q_0: ┤ U1(-π/2) ├┤ U2(0,π) ├┤ U1(-π/2) ├┤ U1(4.8546) ├┤ U1(-π/2) ├┤ U2(0,π) ├»
«     ├──────────┤├─────────┤├──────────┤└─┬────────┬─┘├──────────┤├─────────┤»
«q_1: ┤ U1(-π/2) ├┤

Circuit:
<tket::Circuit, qubits=5, gates=5>


## Example 2


In [8]:
from mqt import qcec

res = qcec.verify(
    "qiskit_circuit_32q_10g_564_52d486_f2d8ef_error_min_qc_qiskit.qasm",
    "qiskit_circuit_32q_10g_564_52d486_f2d8ef_error_min_qc_pytket.qasm")
equivalence = str(res.equivalence)
print(f"Equivalence: {equivalence}")

assert equivalence == "equivalent_up_to_global_phase", "The circuits are not equivalent"
print("The circuits are equivalent")

Equivalence: equivalent_up_to_global_phase
The circuits are equivalent


# Example 3

In [2]:
from mqt import qcec

res = qcec.verify(
    "qiskit_circuit_32q_10g_159_29fa08_4f5c57_error_min_qc_pytket.qasm",
    "qiskit_circuit_32q_10g_159_29fa08_4f5c57_error_min_qc_qiskit.qasm")

equivalence = str(res.equivalence)
print(f"Equivalence: {equivalence}")

Equivalence: not_equivalent


In [3]:
from qiskit.qasm2 import load

circuit_qiskit = load(
    "qiskit_circuit_32q_10g_159_29fa08_4f5c57_error_min_qc_qiskit.qasm")
print(circuit_qiskit)
circuit_pytket = load(
    "qiskit_circuit_32q_10g_159_29fa08_4f5c57_error_min_qc_pytket.qasm")
print(circuit_pytket)

                     ┌─┐                                                      »
 qr_0: ──────────────┤M├──────────────────────────────────────────────────────»
                     └╥┘┌─┐                                                   »
 qr_1: ───────────────╫─┤M├───────────────────────────────────────────────────»
                      ║ └╥┘┌─┐                                                »
 qr_2: ───────────────╫──╫─┤M├────────────────────────────────────────────────»
                      ║  ║ └╥┘┌─┐                                             »
 qr_3: ───────────────╫──╫──╫─┤M├─────────────────────────────────────────────»
                      ║  ║  ║ └╥┘                                             »
 qr_4: ──────■────────╫──╫──╫──╫──────────────────────────────────────────────»
             │        ║  ║  ║  ║ ┌─┐                                          »
 qr_5: ──────┼────────╫──╫──╫──╫─┤M├──────────────────────────────────────────»
             │        ║  ║  ║  ║ └╥┘┌─┐ 

In [36]:
# execute with Statevector
from qiskit.qasm2 import loads
from qiskit import qasm2
from qiskit.quantum_info import Statevector

qiskit_string = """
OPENQASM 2.0;
include "qelib1.inc";
gate dcx q0,q1 { cx q0,q1; cx q1,q0; }
qreg qr[4];
creg cr[4];
crz(4.324221) qr[0],qr[1];
dcx qr[2],qr[3];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
measure qr[2] -> cr[2];
measure qr[3] -> cr[3];
"""
qiskit_circuit = loads(
    qiskit_string, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
print("Original: qiskit")
print(qiskit_circuit)

qiskit_decomposed_circuit = qiskit_circuit.decompose()
print("Decomposed: qiskit")
print(qiskit_decomposed_circuit)


pytket_string = """
OPENQASM 2.0;
include "qelib1.inc";

qreg qr[4];
creg cr[4];
u1(0.6882211471717787*pi) qr[1];
cx qr[2],qr[3];
cx qr[0],qr[1];
cx qr[3],qr[2];
u1(1.3117788528282213*pi) qr[1];
cx qr[0],qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
measure qr[2] -> cr[2];
measure qr[3] -> cr[3];
"""
pytket_circuit = loads(
    pytket_string, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
print("ORIGINAL: pytket")
print(pytket_circuit)


pytket_resorted_string = """
OPENQASM 2.0;
include "qelib1.inc";

qreg qr[4];
creg cr[4];

cx qr[3],qr[2];
cx qr[2],qr[3];

u1(0.6882211471717787*pi) qr[1];
cx qr[0],qr[1];
u1(1.3117788528282213*pi) qr[1];
cx qr[0],qr[1];

measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
measure qr[2] -> cr[2];
measure qr[3] -> cr[3];
"""
pytket_resorted_circuit = loads(
    pytket_resorted_string,
    custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
print("RESORTED: pytket")
print(pytket_resorted_circuit)


# sv_qiskit = Statevector.from_instruction(qiskit_circuit)
# probabilities_qiskit = sv_qiskit.probabilities_dict()
# print(probabilities_qiskit)

# sv_pytket = Statevector.from_instruction(pytket_circuit)
# probabilities_pytket = sv_pytket.probabilities_dict()
# print(probabilities_pytket)

Original: qiskit
                    ┌─┐         
qr_0: ──────■───────┤M├─────────
      ┌─────┴──────┐└╥┘┌─┐      
qr_1: ┤ Rz(4.3242) ├─╫─┤M├──────
      └──┬──────┬──┘ ║ └╥┘┌─┐   
qr_2: ───┤0     ├────╫──╫─┤M├───
         │  Dcx │    ║  ║ └╥┘┌─┐
qr_3: ───┤1     ├────╫──╫──╫─┤M├
         └──────┘    ║  ║  ║ └╥┘
cr: 4/═══════════════╩══╩══╩══╩═
                     0  1  2  3 
Decomposed: qiskit
                                             ┌─┐   
qr_0: ────────────────■───────────────────■──┤M├───
      ┌────────────┐┌─┴─┐┌─────────────┐┌─┴─┐└╥┘┌─┐
qr_1: ┤ Rz(2.1621) ├┤ X ├┤ Rz(-2.1621) ├┤ X ├─╫─┤M├
      └────────────┘├───┤└─────┬─┬─────┘└───┘ ║ └╥┘
qr_2: ──────■───────┤ X ├──────┤M├────────────╫──╫─
          ┌─┴─┐     └─┬─┘      └╥┘       ┌─┐  ║  ║ 
qr_3: ────┤ X ├───────■─────────╫────────┤M├──╫──╫─
          └───┘                 ║        └╥┘  ║  ║ 
cr: 4/══════════════════════════╩═════════╩═══╩══╩═
                                2         3   0  1 
ORIGINAL: pytket
            

In [62]:
from mqt import qcec
import tempfile


def check_two_qasm_strings(str_qiskit, str_pytket):
    # create two temporary files
    with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".qasm") as f_qiskit, tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".qasm") as f_pytket:
        f_qiskit.write(str_qiskit)
        f_pytket.write(str_pytket)
        f_qiskit.flush()
        f_pytket.flush()
        # print(f_qiskit.name)
        # print(f_pytket.name)

        # verify the two files
        res = qcec.verify(f_qiskit.name, f_pytket.name)
        equivalence = str(res.equivalence)
        # print(f"Equivalence: {equivalence}")
        return equivalence

In [32]:
check_two_qasm_strings(
    qiskit_string,
    pytket_string
)

/tmp/tmpqwzl5xh_.qasm
/tmp/tmpnipq_kzh.qasm
Equivalence: equivalent


In [33]:
check_two_qasm_strings(
    qiskit_string,
    pytket_resorted_string
)

/tmp/tmpzazhcu56.qasm
/tmp/tmpttlg2if_.qasm
Equivalence: equivalent


In [5]:
from qiskit.qasm2 import loads
from qiskit import qasm2
from mqt import qcec
import tempfile
from qiskit.quantum_info import Statevector


def check_two_qasm_strings(qasm_str1, qasm_str2):
    # create two temporary files
    with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".qasm") as f1, tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".qasm") as f2:
        f1.write(qasm_str1)
        f2.write(qasm_str2)
        f1.flush()
        f2.flush()
        # verify the two files
        res = qcec.verify(f1.name, f2.name)
        equivalence = str(res.equivalence)
        # print(f"Equivalence: {equivalence}")
        return equivalence


def remove_measurements_and_execute(qasm_str1, qasm_str2):
    # Load the circuits from the QASM strings
    circuit1 = loads(
        qasm_str1, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
    circuit2 = loads(
        qasm_str2, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)

    # Remove measurements from the circuits
    circuit1.remove_final_measurements(inplace=True)
    circuit2.remove_final_measurements(inplace=True)

    # Execute them with statevector simulator
    sv_1 = Statevector.from_instruction(circuit1)
    probabilities_1 = sv_1.probabilities_dict()
    print(qasm_str1)
    print(circuit1)
    print("Statevector simulator result: ", probabilities_1)
    sv_2 = Statevector.from_instruction(circuit2)
    probabilities_2 = sv_2.probabilities_dict()
    print(qasm_str2)
    print(circuit2)
    print("Statevector simulator result: ", probabilities_2)


# minimal example
dcx_str = """
OPENQASM 2.0;
include "qelib1.inc";
gate dcx q0,q1 {
    cx q0,q1;
    cx q1,q0; }
qreg qr[2];
creg cr[2];
x qr[0];
dcx qr[0],qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
"""

decomposed_str = """
OPENQASM 2.0;
include "qelib1.inc";
qreg qr[2];
creg cr[2];
x qr[0];
cx qr[1],qr[0];
cx qr[0],qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
"""

answers = {}
for _ in range(1000):
    res = check_two_qasm_strings(
        dcx_str,
        decomposed_str
    )
    if res in answers.keys():
        answers[res] += 1
    else:
        answers[res] = 1

remove_measurements_and_execute(
    dcx_str,
    decomposed_str
)

answers

The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn.
The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn.
The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn.
The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn.
The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn.
The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn


OPENQASM 2.0;
include "qelib1.inc";
gate dcx q0,q1 {
    cx q0,q1;
    cx q1,q0; }
qreg qr[2];
creg cr[2];
x qr[0];
dcx qr[0],qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];

      ┌───┐┌──────┐
qr_0: ┤ X ├┤0     ├
      └───┘│  Dcx │
qr_1: ─────┤1     ├
           └──────┘
Statevector simulator result:  {'10': 1.0}

OPENQASM 2.0;
include "qelib1.inc";
qreg qr[2];
creg cr[2];
x qr[0];
cx qr[1],qr[0];
cx qr[0],qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];

      ┌───┐┌───┐     
qr_0: ┤ X ├┤ X ├──■──
      └───┘└─┬─┘┌─┴─┐
qr_1: ───────■──┤ X ├
                └───┘
Statevector simulator result:  {'11': 1.0}


{'equivalent': 966, 'no_information': 34}

In [ ]:

# Example usage
qasm_str1 = """
OPENQASM 2.0;
include "qelib1.inc";
qreg qr[2];
creg cr[2];
x qr[0];
measure qr[0] -> cr[0];
"""

qasm_str2 = """
OPENQASM 2.0;
include "qelib1.inc";
qreg qr[2];
creg cr[2];
x qr[1];
measure qr[1] -> cr[1];
"""

remove_measurements_and_execute(qasm_str1, qasm_str2)

In [66]:
# minimal example

dcx_str = """
OPENQASM 2.0;
include "qelib1.inc";
gate dcx q0,q1 {
    cx q0,q1;
    cx q1,q0; }
qreg qr[2];
creg cr[2];
x qr[0];
dcx qr[0],qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
"""

decomposed_str = """
OPENQASM 2.0;
include "qelib1.inc";
qreg qr[2];
creg cr[2];
x qr[0];
cx qr[1],qr[0];
cx qr[0],qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
"""

answers = {}
for _ in range(1000):
    res = check_two_qasm_strings(
        dcx_str,
        decomposed_str
    )
    if res in answers.keys():
        answers[res] += 1
    else:
        answers[res] = 1

remove_measurements_and_execute(
    dcx_str,
    decomposed_str
)

answers

The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn.
The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn.
The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn.
The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn.
The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn.
The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn


OPENQASM 2.0;
include "qelib1.inc";
gate dcx q0,q1 { 
    cx q0,q1; 
    cx q1,q0; }
qreg qr[2];
creg cr[2];
x qr[0];
dcx qr[0],qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];

      ┌───┐┌──────┐
qr_0: ┤ X ├┤0     ├
      └───┘│  Dcx │
qr_1: ─────┤1     ├
           └──────┘
{'10': 1.0}

OPENQASM 2.0;
include "qelib1.inc";
qreg qr[2];
creg cr[2];
x qr[0];
cx qr[1],qr[0];
cx qr[0],qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];

      ┌───┐┌───┐     
qr_0: ┤ X ├┤ X ├──■──
      └───┘└─┬─┘┌─┴─┐
qr_1: ───────■──┤ X ├
                └───┘
{'11': 1.0}


The ZX checker suggests that the circuits are not equivalent, but the simulation checker suggests that they are probably equivalent. Thus, no conclusion can be drawn.


{'equivalent': 978, 'no_information': 22}

In [42]:
qiskit_string = """
OPENQASM 2.0;
include "qelib1.inc";
gate dcx q0,q1 { cx q0,q1; cx q1,q0; }
qreg qr[4];
creg cr[4];
dcx qr[0],qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
measure qr[2] -> cr[2];
measure qr[3] -> cr[3];
"""

pytket_resorted_string = """
OPENQASM 2.0;
include "qelib1.inc";

qreg qr[4];
creg cr[4];

cx qr[0],qr[1];
cx qr[1],qr[0];


measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
measure qr[2] -> cr[2];
measure qr[3] -> cr[3];
"""

check_two_qasm_strings(
    qiskit_string,
    pytket_resorted_string
)

/tmp/tmpt0nv84ks.qasm
/tmp/tmphr7ds5ni.qasm
Equivalence: equivalent


In [1]:
from mqt import qcec
import qiskit
print(qiskit.__version__)

print(qcec.__version__)

1.2.4
2.7.1
